In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName('Partitions')
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 05:06:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 사용자 지정 파티션 생성하기


In [3]:
pairs = sc.parallelize([
    1,2,3,4,2,4,1
]).map(lambda x : (x, x))

pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]

In [4]:
pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [5]:
# 개발자가 직접 파티션 배치 함수를 만들 수 있다.
pairs.partitionBy(2, lambda x : x % 2).glom().collect()
# 나머지가 같은 것, 비슷한 것 끼리 파티션 만들어짐

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [6]:
# hash argorithm 의 기본

파티션을 만든 후에 persist()를 하지 않으면 다음 연산에 불릴 때 마다 파티션을 만드는 코드가 계속 반복
- 셔플링이 반복적으로 계속 일어난다.

In [7]:
pairs.partitionBy(2, lambda x : x % 2).persist().glom().collect()
# 계속 실행해도 파티션을 다시 나누지 않는다.

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 35118)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/spark_env/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/miniconda3/envs/spark_env/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/miniconda3/envs/spark_env/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ubuntu/miniconda3/envs/spark_env/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ubuntu/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/home/ubuntu/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/accumulators.